In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

from  bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys 
import time

import warnings
warnings.filterwarnings('ignore')

import string, nltk, re
from konlpy.tag import Okt
from wordcloud import WordCloud

from PIL import Image
from wordcloud import ImageColorGenerator

import os
driver_name =  os.environ['HOMEDRIVE'] + os.environ['HOMEPATH'] + '\\Downloads\\chromedriver_win32\\chromedriver.exe'

### 영화관입장권통합전산망 KOBIS 에서 영화 정보 크롤링

In [27]:
url = "https://www.kobis.or.kr/kobis/business/mast/mvie/searchMovieList.do"

home_url = "https://www.kobis.or.kr/"

In [44]:
driver = webdriver.Chrome(driver_name)

driver.get(url)
time.sleep(2)

# paging
lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')

end, current = len(lis), 0
while True:

    trs = driver.find_elements(By.CSS_SELECTOR, 'tbody > tr')

    lines = []
    for tr in trs:
        td = tr.find_elements(By.TAG_NAME, 'td')

        kind = td[5].text
        genre = td[6].text
        
        if kind =='' or kind =='단편' or kind == "기타" or kind == '옴니버스':
            continue
        if '성인' in genre or genre == '' :
            continue

        td[0].find_element(By.TAG_NAME, 'a').click()
        time.sleep(1)

        # 영화 제목
        title = driver.find_element(By.CSS_SELECTOR, '.hd_layer > div').text
        find_index = title.find(')') 
        if find_index >= 0 :
            title = title[:find_index+1]
        
        infos = driver.find_element(By.CSS_SELECTOR, 'div.basic')

        # 시놉시스가 없으면 pass
        try :
            synopsis = infos.find_element(By.CSS_SELECTOR, '.info.info2 > pp').text
            print(synopsis)
        except :
            driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()
            continue

        infos_head = infos.find_element(By.CSS_SELECTOR, '.ovf.info.info1')
            
        # 이미지 URL
        img = infos_head.find_element(By.CSS_SELECTOR, 'a > img').get_attribute('src')
        # 요약 정보
        dds = infos_head.find_elements(By.CSS_SELECTOR, 'dl > dd')
        summering = dds[3].text
        # 개봉일
        first_day = dds[5].text
        # 제작년도
        production_year = dds[6].text
    
        
        # 감독 / 주연 배우
        tmp = infos.find_elements(By.CSS_SELECTOR, '.info.info2 > .staffMore > dl > div > dd')
        movie_director, star_actor = '', ''
        if len(tmp) >= 1:
            movie_director = tmp[0].text
        if len(tmp) >= 2:
            tmp2 = tmp[1].find_elements(By.TAG_NAME, 'table')
            
            if len(tmp2) >= 1:
                star_actor = tmp2[0].text
            else:
                star_actor = tmp[1].text
            star_actor = star_actor.replace('[주연] ', '')
            print(star_actor)

        lines.append({'title': title, 'genre': genre, 'summering': summering, 
                    'first_day': first_day, 'production_year': production_year, 
                    'movie_director':movie_director, 'star_actor': star_actor, 
                    'img': img, 'synopsis': synopsis})
        
        driver.find_element(By.CSS_SELECTOR, '.hd_layer > a.close').click()


    current += 1
    if end == current:
        try:
            driver.find_element(By.CSS_SELECTOR, '.paging > a.btn.next').click()
            time.sleep(1)
            lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
            end, current = len(lis), 0
        except:
            break
    else:
        # 다음 페이지 넘어가기
        lis = driver.find_elements(By.CSS_SELECTOR, '.paging > ul > li')
        lis[current].click()
        time.sleep(2)


KeyboardInterrupt: 

In [45]:
lines

[]